In [1]:
import os
import numpy as np
from skimage.io import imsave, imread
import cv2

In [2]:
dim=513
data_path=r'C:\Users\nishc_omjn2ty\Downloads\SAMPLES_SINGLE_CLASS\TO_VINOD_V2_12OCT2020'

In [3]:
def create_train_data():
    train_data_path = os.path.join(data_path, 'RASTER')
    train_data_path_mask = os.path.join(data_path, 'MASKS')
    images = os.listdir(train_data_path)
    total = len(images)

    imgs = []
    imgs_mask = []

    i = 0
    print('-'*30)
    print('Creating training images...')
    print('-'*30)
    for image_name in images:
        image_name_mask = image_name.split('.')[0] + '.png'
        img = imread(os.path.join(train_data_path, image_name))
        img_mask = imread(os.path.join(train_data_path_mask, image_name_mask), as_grey=True)
        img = np.array(img)
        img_mask = np.array(img_mask)
        img = cv2.resize(img, (256, 256))
        img_mask = cv2.resize(img_mask, (256, 256))
        imgs.append(img)
        imgs_mask.append(img_mask)

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    #print(imgs[0])
    imgs= np.array(imgs)
    imgs_mask=np.array(imgs_mask)
    imgs_mask=imgs_mask[..., np.newaxis]
    return imgs,imgs_mask


In [4]:
def load_train_data():
    return create_train_data()

In [5]:
a,b = load_train_data()

------------------------------
Creating training images...
------------------------------


C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\skimage\io\_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


Done: 0/415 images
Done: 100/415 images
Done: 200/415 images
Done: 300/415 images
Done: 400/415 images
Loading done.


In [8]:
a = a.astype('float32')
mean = np.mean(a)  # mean for data centering
std = np.std(a)  # std for data normalization

a -= mean
a /= std
a = a.astype('float32')

In [9]:
b = b.astype('float32')
b/= 255.

# TRAINING

In [6]:
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Flatten, Dense, BatchNormalization
from keras.layers.core import Dropout
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [7]:
import tensorflow as tf

In [10]:
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
img_rows = 256
img_cols = 256
smooth = 1.

In [11]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols, imgs.shape[3]))
    for i in range(imgs.shape[0]):
        imgs_p[i] = cv2.resize(imgs[i], (img_cols, img_rows))

    #imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [12]:
def DiceBCELoss(targets, inputs, smooth=1e-6):    
       
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    binary_crossentropy = tf.keras.losses.BinaryCrossentropy()
    BCE =  binary_crossentropy(targets, inputs)
    intersection = K.sum(targets*inputs)   
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    Dice_BCE = BCE + dice_loss
    
    return Dice_BCE

# Model Similar to UNET

In [13]:
def get_model():
    inputs = Input((img_rows, img_cols, 3))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)
    
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv6), conv5], axis=3)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv7), conv4], axis=3)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv8), conv3], axis=3)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv9)

    up10 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv9), conv2], axis=3)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv10)

    up11 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv10), conv1], axis=3)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid')(conv11)

    model = Model(inputs=[inputs], outputs=[conv12])

    model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [14]:
model = get_model()
model.summary()

W1019 21:38:02.460470 19060 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1019 21:38:03.135401 19060 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1019 21:38:03.236949 19060 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1019 21:38:03.507253 19060 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1019 21:38:04.769695 19060 deprecation_wrapper

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 32) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [15]:
def train_and_save():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train, imgs_mask_train = load_train_data()
    
    print('Preprocessing Done...')
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_model()
    #model.load_weights('weights.h5')
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=200, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

#     model.fit_generator(train_generator, steps_per_epoch = (332//32), epochs=400, verbose=1, shuffle=True,
#               validation_data=val_generator,
#               validation_steps=(83//32), 
#               callbacks=[model_checkpoint])


In [16]:
model = get_model()
model.load_weights('weights.h5')

W1019 21:38:17.783773 19060 deprecation_wrapper.py:119] From C:\Users\nishc_omjn2ty\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [19]:
def test_and_predict(imgs_test):
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    #imgs_test = imread(os.path.join(data_path, 'RASTER/TOF_CHIPS91.jpg'))
    imgs_test = cv2.resize(imgs_test, (img_cols, img_rows))
    imgs_test = np.array([imgs_test])

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    image = (imgs_mask_test[0] * 255.).astype(np.uint8)
    imsave(str(1) + '_pred.png', image)
    return image


In [ ]:
train_and_save()

In [20]:
import cv2

imgs_test = imread(os.path.join(data_path, 'MASKS/TOF_CHIPS91.png'), as_grey=True)
test = imread(os.path.join(data_path, 'RASTER/TOF_CHIPS91.JPG'))
chip = test_and_predict(test)
chip=cv2.threshold(chip,127,255,cv2.THRESH_BINARY)
#imgs_test = resize(imgs_test, (img_cols, img_rows))
cv2.imshow("predicted",chip[1])
cv2.imshow("ACTUAL",test)
cv2.waitKey(0)

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 12s 12s/step


-1

In [21]:
hrs = imread('hrs_data_lut.tif')

In [132]:
import matplotlib.pyplot as plt
cv2.imshow('img',y)
cv2.waitKey(0)

-1

In [135]:
hrs.shape

(6719, 9872, 3)

In [151]:
hrs_m= cv2.copyMakeBorder(hrs, 0,193, 0,112, cv2.BORDER_CONSTANT)

In [153]:
hrs_m.shape

(6912, 9984, 3)

In [ ]:
x=image[1]
masks=[[]]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            masks=x
        else:
            masks= np.vstack((masks,x))
        imgs_mask_test = model.predict(np.array([a[i]]), verbose=1)
        img = (imgs_mask_test[0] * 255.).astype(np.uint8)
        #img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        x=img[1]
        continue
    imgs_mask_test = model.predict(np.array([a[i]]), verbose=1)
    img = (imgs_mask_test[0] * 255.).astype(np.uint8)
    img=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    x = np.hstack((x,img[1]))
    
    

In [ ]:
x=a[0]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            y=x
        else:
            y= np.vstack((y,x))
        x=a[i]
        continue
    x = np.hstack((x,a[i]))

In [ ]:
x=b[0]
orig=[[]]
for i in range(1,400):
    if(i%20==0):
        if(i==20):
            orig=x
        else:
            orig= np.vstack((orig,x))
        x=b[i]
        continue
    x = np.hstack((x,b[i]))

In [ ]:
imsave('PREDICTED_MASKS.png',masks)
imsave('ORIGINAL_MASKS.png',orig)
imsave('RGB.png',y)

In [ ]:
x=a[0]
y_1=[[]]
for i in range(1,400):
    if(i%13==0):
        if(i==13):
            y_1=x
        else:
            y_1= np.vstack((y_1,x))
        x=a[i]
        continue
    x = np.hstack((x,a[i]))

In [154]:
#Function for doing it on large image
for i in range(0,6912,256):
    x=[[]]
    for j in range(0,9984,256):
        imgs_test = hrs_m[i:i+256, j:j+256, :3]
        imgs_mask_test = test_and_predict(imgs_test)
        img =  cv2.threshold(imgs_mask_test,127,255,cv2.THRESH_BINARY)
        if(j==0):
            x=img[1]
            #x=imgs_mask_test
            continue
        x = cv2.hconcat([x[:,:-1],x[:,-2],img[1][:,1],img[1][:,1:]])
    if(i==0):
        y=x
    else:
        y = np.vstack((y[:-1,:],y[-2,:],x[1,:],x[1:,:]))

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 339ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 44ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 60ms

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 46ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 49ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 51ms/

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 102ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 41ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 43ms

1/1 [==============================] - 0s 104ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 120ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 111ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
-----------------------

------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 191ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 106ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 118ms/step
------------------------------
Loading and preprocessing test data...
-----------------------

...

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 103ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 162ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 116

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 106ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 116ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 134

1/1 [==============================] - 0s 110ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 102ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 104ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
-----------------------

1/1 [==============================] - 0s 114ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 107ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 108ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
-----------------------

------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 115ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 124ms/step
------------------------------
Loading and preprocessing test data...
------------------------------
------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
1/1 [==============================] - 0s 116

In [161]:
imsave('PREDICTED_MASKS_HRS_output.tif',y_mod)

In [34]:
hrs.shape

(6719, 9872, 3)

In [157]:
y_mod = y[:6719, :9872]

In [162]:
y_mod.shape

(6719, 9872)